# Testes da rotina da classe do Sistema de Lorenz

## 1. Importando as bibliotecas necessárias

### Bibliotecas obrigatórias

In [111]:
import numpy as np
from scipy.integrate import odeint

%matplotlib widget
import matplotlib.pyplot as plt

import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 100

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

plt.style.use('dark_background')

### Bibliotecas não-obrigatórias

In [112]:
import sys 
sys.path.insert(0, '../../scripts')
import lorenzsystem as lsys

## 2. Definindo a função (debug)

Não é necessário executar essa linha se você estiver importando o arquivo com a classe.

In [170]:
class SistemaLorenz:
    
    def __init__(self, sigma=10, beta=8/3, rho=28):
        """
        Descrição:
        ----------
        Construtor da classe 'SistemaLorenz'

        Parâmetros:
        -----------
        sigma: int ou float
            Parâmetro do Sistema de Lorenz
        beta: int ou float
            Parâmetro do Sistema de Lorenz
        rho: int ou float
            Parâmetro do Sistema de Lorenz
        estado_inicial: np.ndarray
            Vetor das posições xyz iniciais do sistema
        """
        
        if not (((type(sigma) is int) | (type(sigma) is float)) & (sigma > 0)):
            raise TypeError("Sigma deve ser um int ou float positivo!")

        if not (((type(beta) is int) | (type(beta) is float)) & (beta >= 0)):
            raise TypeError("Beta deve ser um int ou float positivo!")

        if not (((type(rho) is int) | (type(rho) is float)) & (rho >= 0)):
            raise TypeError("Rho deve ser um int ou float positivo!")
        
        self._sigma = sigma
        self._beta = beta
        self._rho = rho
        pass
    
    def equacoes(self, estado_atual, t):
        """
        Descrição:
        ----------
        Retorna as equações do Sistema de Lorenz calculadas no instante t atual
        
        Parâmetros:
        -----------
        estado_inicial: np.ndarray
            Vetor das posições xyz atuais do sistema
        """
        
        if not (type(estado_atual) is np.ndarray):
            raise TypeError("O vetor estado atual deve ser um array do numpy!")        
        
        sigma = self._sigma
        beta = self._beta
        rho = self._rho
        x, y, z = estado_atual
        
        dx_dt = sigma * (y - x)
        dy_dt = x * (rho - z) - y
        dz_dt = x * y - beta * z
        return [dx_dt, dy_dt, dz_dt]

## 3. Testando com o atrator de Lorenz

In [179]:
t_inicial = 0
t_final = 100
instantes_temporais = np.linspace(t_inicial, t_final, (t_final - t_inicial)*1000)

In [180]:
sis_lorenz = SistemaLorenz()

In [181]:
estado_inicial = 0.1, 0, 0

In [182]:
xyz = odeint(func=sis_lorenz.equacoes, t=instantes_temporais, y0=estado_inicial)

In [183]:
x = xyz[:, 0]
y = xyz[:, 1]
z = xyz[:, 2]

### 3.a) Diagrama de Fases

In [184]:
fig, ax = plt.subplots()
ax = plt.axes(projection='3d')
ax.plot(x, y, z, alpha=1, linewidth=0.6, color='White')
ax.set_title("Diagrama de fase do Atrator de Lorenz")
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

ax.w_xaxis.set_pane_color((0, 0, 0, 0))
ax.w_yaxis.set_pane_color((0, 0, 0, 0))
ax.w_zaxis.set_pane_color((0, 0, 0, 0))

ax.grid(False)
sns.despine()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 3.b) Séries Temporais

In [185]:
fig, ax = plt.subplots(3)
fig.suptitle("Séries temporais de 0 a 100 segundos das coordenadas xyz do Sistema de Lorenz")
ax[0].plot(instantes_temporais, x, color='DeepSkyBlue')

ax[0].set_ylabel('x(t)')
ax[0].set_xlabel('t')
ax[0].set_xlim(0,)
    
ax[0].grid(True)

ax[1].plot(instantes_temporais, y, color='LightCoral')
ax[1].set_ylabel('y(t)')
ax[1].set_xlabel('t')
ax[1].set_xlim(0,)
ax[1].grid(True)

ax[2].plot(instantes_temporais, z, color='PaleGreen')
ax[2].set_ylabel('z(t)')
ax[2].set_xlabel('t')
ax[2].set_xlim(0,)
ax[2].grid(True)

fig.tight_layout()
sns.despine()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …